In [8]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [16]:

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'
users_filename = 'BX-Users.csv'

In [113]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

df_users = pd.read_csv(
    users_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'location'],
    usecols=['user', 'location'],
    dtype={'user': 'int32', 'location': 'str'})


In [114]:
user_ratings = df_ratings.groupby('user', as_index = False)['rating'].count().rename(columns = { 'rating':'usercount'})
book_ratings = df_ratings.groupby('isbn', as_index = False)['rating'].count().rename(columns = {'rating':'titlecount'})
df_ratings = pd.merge(left = df_ratings, right = user_ratings, on = 'user')
df_ratings = pd.merge(left = df_ratings, right = book_ratings, on = 'isbn')

df = df_ratings.loc[(df_ratings['usercount'] >= 200) & (df_ratings['titlecount'] >= 100)]
df = df.drop(['usercount', 'titlecount'], axis = 1)
df = pd.merge(left = df, right = df_books, on = 'isbn')

In [115]:
df = df.drop_duplicates(['user', 'title'])

In [116]:
df

,user,isbn,rating,title,author
0,278418,0446520802,0.0,The Notebook,Nicholas Sparks
1,3363,0446520802,0.0,The Notebook,Nicholas Sparks
2,7158,0446520802,10.0,The Notebook,Nicholas Sparks
3,11676,0446520802,10.0,The Notebook,Nicholas Sparks
4,23768,0446520802,6.0,The Notebook,Nicholas Sparks
...,...,...,...,...,...
49512,252071,0312971125,0.0,Crazy for You,Jennifer Crusie
49513,258185,0312971125,0.0,Crazy for You,Jennifer Crusie
49514,263877,0312971125,0.0,Crazy for You,Jennifer Crusie
49515,265115,0312971125,10.0,Crazy for You,Jennifer Crusie


In [117]:
df_pivot = pd.pivot(df, values = 'rating', index = 'title', columns = 'user' )

In [118]:
df_pivot = df_pivot.fillna(0)

In [ ]:
df_pivot

In [120]:
import scipy as spy
df_matrix = spy.sparse.csr_matrix(df_pivot.values)

In [121]:
model = NearestNeighbors(algorithm = 'brute', metric = 'cosine')
model.fit(df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [124]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  x = df_pivot[df_pivot.index == book]
  dist, ind = model.kneighbors(x, n_neighbors = 6)
  recommended_books = []
  reco_books = []
  dist = dist.flatten()
  ind = ind.flatten()
  for i in range(len(ind)):
      if i == 0:
        recommended_books.append(df_pivot.index[ind[i]])
      else:
        reco_book = df_pivot.index[ind[i]]
        reco_dist = dist[i]
        reco_books.append([reco_book, reco_dist])
  #reversing the list order because according to forums, the test function has a bug
  reco_books = reco_books[::-1]
  recommended_books.append(reco_books)
        


  return recommended_books

In [126]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
